### Import relevant packages

In [ ]:
import requests
import pandas as pd
import numpy as np
import yfinance as yf
import time
from datetime import datetime
from datetime import timedelta
from datetime import date

In [ ]:
response = requests.get("https://house-stock-watcher-data.s3-us-west-2.amazonaws.com/data/all_transactions.json")

In [ ]:
dic = response.json()
df = pd.DataFrame(dic)
df['ticker'] = df['ticker'].replace({'BLFSD':'BLFS'})                                         
df.head()

In [ ]:
Nancy_pelosi = df[df['representative'] == "Hon. Nancy Pelosi"]
Nancy_pelosi[Nancy_pelosi['amount'] == '$1,000,001 - $5,000,000'].head()

In [ ]:
purch_dates = df[df["type"]=="purchase"].groupby(['transaction_date', 'ticker']).size().reset_index(name="freq")
purch_dates = purch_dates[purch_dates["ticker"]!="--"].reset_index(drop=True)
purch_dates

In [ ]:
for i in range(len(purch_dates)):
    date_time_obj = datetime.strptime(purch_dates.loc[i,'transaction_date'], '%Y-%m-%d')
    NextDay_date  = date_time_obj + timedelta(days=1)
    his1mon_date  = date_time_obj - timedelta(days=30)
    his3mon_date  = date_time_obj - timedelta(days=90)
    his6mon_date  = date_time_obj - timedelta(days=180)
    his9mon_date  = date_time_obj - timedelta(days=270)
    his12mon_date = date_time_obj - timedelta(days=365)
    purch_dates.loc[i, "trans_date"]    = date_time_obj
    purch_dates.loc[i, "next_date"]     = NextDay_date
    purch_dates.loc[i, "his1mon_date"]  = his1mon_date
    purch_dates.loc[i, "his3mon_date"]  = his3mon_date
    purch_dates.loc[i, "his6mon_date"]  = his6mon_date
    purch_dates.loc[i, "his9mon_date"]  = his9mon_date
    purch_dates.loc[i, "his12mon_date"] = his12mon_date
    
    his1mon_date_nxt  = date_time_obj - timedelta(days=29)
    his3mon_date_nxt  = date_time_obj - timedelta(days=89)
    his6mon_date_nxt  = date_time_obj - timedelta(days=179)
    his9mon_date_nxt  = date_time_obj - timedelta(days=269)
    his12mon_date_nxt = date_time_obj - timedelta(days=364)
    
    purch_dates.loc[i, "his1mon_date_nxt"]  = his1mon_date_nxt
    purch_dates.loc[i, "his3mon_date_nxt"]  = his3mon_date_nxt
    purch_dates.loc[i, "his6mon_date_nxt"]  = his6mon_date_nxt
    purch_dates.loc[i, "his9mon_date_nxt"]  = his9mon_date_nxt
    purch_dates.loc[i, "his12mon_date_nxt"] = his12mon_date_nxt

    
purch_dates

In [ ]:
day_price = pd.DataFrame(columns=["ticker", "date","price"])

for i in range(len(purch_dates)):
    ticker = yf.Ticker(purch_dates.loc[i,"ticker"])
    time.sleep(0.5)
    val = ticker.history(start=purch_dates.loc[i,"trans_date"],
                         end  =purch_dates.loc[i,"next_date"])
    print(i,". ", purch_dates.loc[i,"ticker"], "\t",val[["Open","Close"]],"\n\n")
    try:
        row = pd.DataFrame({"ticker":[purch_dates.loc[i,"ticker"]],
                            "date":[purch_dates.loc[i,"transaction_date"]],
                            "price":[val["Close"].values[0]]})
    except:
        row = pd.DataFrame({"ticker":[purch_dates.loc[i,"ticker"]],
                            "date":[purch_dates.loc[i,"transaction_date"]],
                            "price":[np.nan]})
    day_price = pd.concat([day_price,row], axis = 0)

day_price.to_csv("day_price.csv")

In [ ]:
current_price = pd.DataFrame(columns=["ticker", "date","price"])

for i in range(len(purch_dates)):
    ticker = yf.Ticker(purch_dates.loc[i,"ticker"])
    time.sleep(0.5)
    val = ticker.history(start=date.today().strftime("%Y-%m-%d"))
    print(i,". ", purch_dates.loc[i,"ticker"], "\t",val[["Open","Close"]],"\n\n")
    try:
        row = pd.DataFrame({"ticker":[purch_dates.loc[i,"ticker"]],
                            "date":[date.today().strftime("%Y-%m-%d")],
                            "price":[val["Close"].values[0]]})
    except:
        row = pd.DataFrame({"ticker":[purch_dates.loc[i,"ticker"]],
                            "date":[date.today().strftime("%Y-%m-%d")],
                            "price":[np.nan]})
    current_price = pd.concat([current_price,row], axis = 0)

current_price.to_csv("current_price.csv")    

In [ ]:
'''
his1_price = pd.DataFrame(columns=["ticker", "date","price"])

for i in range(len(purch_dates)):
    ticker = yf.Ticker(purch_dates.loc[i,"ticker"])
    time.sleep(0.5)
    val = ticker.history(start=purch_dates.loc[i,"his1mon_date"],
                         end  =purch_dates.loc[i,"his1mon_date_nxt"])
    print(i,". ", purch_dates.loc[i,"ticker"], "\t",val[["Open","Close"]],"\n\n")
    try:
        row = pd.DataFrame({"ticker":[purch_dates.loc[i,"ticker"]],
                            "date":[purch_dates.loc[i,"his1mon_date"]],
                            "price":[val["Close"].values[0]]})
    except:
        row = pd.DataFrame({"ticker":[purch_dates.loc[i,"ticker"]],
                            "date":[purch_dates.loc[i,"his1mon_date"]],
                            "price":[np.nan]})
    his1_price = pd.concat([his1_price,row], axis = 0)
'''

### Add columns and modify the df for classification

In [ ]:
#Add transaction day stock-price
new_df = pd.merge(df, 
                  day_price[~np.isnan(day_price["price"])],
                  how='inner', 
                  left_on=['ticker','transaction_date'],
                  right_on = ['ticker','date'])

new_df = new_df.rename(columns={'price': 'trans_price'})

#Add current stock-price
new_df = pd.merge(new_df, 
                  current_price[~np.isnan(current_price["price"])],
                  how='inner', 
                  left_on=['ticker'],
                  right_on = ['ticker'])

new_df = new_df.rename(columns={'price': 'current_price'})

#Add Flag for response (if current price > trans_day_price)
new_df['response'] = new_df['current_price'] > new_df['trans_price']

#Filter for only purchase transactions
new_df = new_df[new_df["type"]=="purchase"]

#Drop unnecessary columns
new_df = new_df.drop(columns=['transaction_date',
                              'disclosure_date',
                              'disclosure_year',
                              'cap_gains_over_200_usd',
                              'asset_description',
                              'ptr_link',
                              'date_x',
                              'date_y',
                              'district'])

#Create 1-hot-encoded columns
new_df = pd.get_dummies(new_df)

new_df.to_csv("new_df.csv")    

In [ ]:
for i in range(len(new_df.columns)): print(i)

### Build model for classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

X,y = new_df.drop(columns=['response']), 1*new_df['response']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)

#clf = RandomForestClassifier(max_depth=2, random_state=0)
#clf = LogisticRegression(random_state=0, max_iter=1000)
clf  = tree.DecisionTreeClassifier()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Test set accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cmd = ConfusionMatrixDisplay(cm)
cmd.plot()

In [ ]:
new_pred = clf.predict(X_train)
accuracy_score(y_train, new_pred)

In [ ]:
new_df.corr()